In [1]:
import pandas as pd

In [2]:
policy_filepath = "policy_database.xlsx"
codebook_filepath = "policy_codebook.xlsx"

policies = pd.read_excel(policy_filepath)
codebook = pd.read_excel(codebook_filepath)

In [3]:
policies[:5]

,state,year,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,...,expartedating,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal
0,Alabama,1991,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,15
1,Alabama,1992,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,16
2,Alabama,1993,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,16
3,Alabama,1994,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,16
4,Alabama,1995,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,16


In [4]:
codebook[:5]

,Category Code,Category,Sub-Category,Variable Name,Brief Description of Provision,Detailed Description of Provision,Coding Notes,Coding Instructions,Notes,Data Source and Attribution
0,1,Dealer regulations,Licensing,dealer,State dealer license required for sale of all ...,All firearm dealers are required to have a sta...,State requires all persons engaged in the busi...,If all firearm dealers are required to have a ...,NaN,"Coded by Michael Siegel, MD, MPH, Boston Unive..."
1,1,Dealer regulations,Licensing,dealerh,State dealer license required for sale of hand...,All firearm dealers that sell handguns are req...,State requires all persons engaged in the busi...,If only dealers of handguns or handguns AND as...,NaN,"Coded by Michael Siegel, MD, MPH, Boston Unive..."
2,1,Dealer regulations,Recordkeeping,recordsdealer,Licensed dealers are required to keep and reta...,Licensed dealers are required to keep and reta...,Recordkeeping refers to the recording of sales...,If record keeping and retention is required on...,Federal law requires licensed dealers to keep ...,"Coded by Michael Siegel, MD, MPH, Boston Unive..."
3,1,Dealer regulations,Recordkeeping,recordsdealerh,Licensed dealers are required to keep and reta...,Licensed dealers are required to keep and reta...,Recordkeeping refers to the recording of sales...,If record keeping and retention is required on...,Federal law requires licensed dealers to keep ...,"Coded by Michael Siegel, MD, MPH, Boston Unive..."
4,1,Dealer regulations,Recordkeeping,recordsall,All private sellers and licensed dealers are r...,Both private sellers and licensed dealers are ...,Recordkeeping refers to the recording of sales...,If all private sellers and licensed dealers ar...,Federal law requires licensed dealers to keep ...,"Coded by Michael Siegel, MD, MPH, Boston Unive..."


In [5]:
sub_categories = {}
for category in set(codebook['Sub-Category']):
    sub_categories[category] = list(codebook[codebook['Sub-Category'] == category]['Variable Name'])

In [17]:
metadata = {'year':[], 'state':[], 'category':[], 'sub_category':[], 'policies_implemented':[]}
for i in range(len(policies)):
    for j, (sub_category, variables) in enumerate(sub_categories.items()):
        metadata['year'].append(policies.year[i])
        metadata['state'].append(policies.state[i])
        metadata['category'].append(codebook[codebook['Sub-Category'] == sub_category].Category.iloc[0])
        metadata['sub_category'].append(sub_category)
        policies_implemented = 0
        for variable in variables:
            policies_implemented += policies[variable][i]
        metadata['policies_implemented'].append(policies_implemented)

In [18]:
metadata_df = pd.DataFrame(metadata)
metadata_df.sort_values(['year', 'state', 'category'], inplace=True)
metadata_df

,year,state,category,sub_category,policies_implemented
11,1991,Alabama,Ammunition regulations,Prohibitors,0
38,1991,Alabama,Ammunition regulations,Background checks,0
28,1991,Alabama,Assault weapons and large-capacity magazines,Assault weapons ban,0
46,1991,Alabama,Assault weapons and large-capacity magazines,Large capacity magazine ban,0
22,1991,Alabama,Background checks,Universal background checks,0
...,...,...,...,...,...
74965,2020,Wyoming,Prohibitions for high-risk gun possession,Felony,1
74967,2020,Wyoming,Prohibitions for high-risk gun possession,Relinquishment of weapons,0
74973,2020,Wyoming,Prohibitions for high-risk gun possession,Drugs,0
74993,2020,Wyoming,Prohibitions for high-risk gun possession,Mental Health,0


In [19]:
metadata_df[metadata_df.policies_implemented > 0]

,year,state,category,sub_category,policies_implemented
18,1991,Alabama,Buyer regulations,Permitting,3
49,1991,Alabama,Buyer regulations,Waiting period,1
6,1991,Alabama,Dealer regulations,Licensing,1
8,1991,Alabama,Dealer regulations,Recordkeeping,2
13,1991,Alabama,Dealer regulations,Security,1
...,...,...,...,...,...
74990,2020,Wyoming,Buyer regulations,Age restrictions,2
74982,2020,Wyoming,Immunity,Immunity,1
74981,2020,Wyoming,Possession regulations,School zones,1
74994,2020,Wyoming,Possession regulations,Campus carry,2


In [22]:
metadata_df.to_csv('policy_metadata.csv', index=False)